In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
import matplotlib.patches as mpatches

%matplotlib inline

In [11]:
rna_file = '/data/katrina/AutoEncode/scdata/b_cells_human_counts.norm.csv'#rsv_human_counts.norm.csv' # DO NOT USE SCALED DATA

rna_out_file = os.path.join('Data', 'b_cells_human_counts.norm.mad.csv.gz')
rna_out_zeroone_file = os.path.join('Data', 'b_cells_human_counts.norm.mad.csv.gz')

In [12]:
rnaseq_df = pd.read_table(rna_file, index_col=0,sep=',')
rnaseq_df.head()

,CGCAGGACTCTGGA,TCATTCGATCAGAC,GAGCAACTGCCAAT,CTTAACACCATACG,CAACAGACTAGACC,GCTCAGCTTGCCCT,CTACGCACGTATCG,TGACTTTGGATAGA,TGTCTAACGGTTCA,TATCTCGATCTAGG,...,GTATGGTGCGACTA,CAGTTTACACCCAA,TGCTAGGAAATCGC,GCTACCTGGTCACA,GGACTATGAAAAGC,AACACTCTGTTACG,GCAAGACTGGGACA,ATTCGGGAAGTTCG,GGTGGAGATAGAGA,ACGTCAGACATGAC
AL627309.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RP11-206L10.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RP11-206L10.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LINC00115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM41C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Process RNAseq file
#rnaseq_df.index = rnaseq_df.index.map(lambda x: x.split('|')[0])
#rnaseq_df.columns = rnaseq_df.columns.str.slice(start=0, stop=15)

# Gene is listed twice in RNAseq data, drop both occurrences
#rnaseq_df.drop('SLC35E2', axis=0, inplace=True)
rnaseq_df = rnaseq_df.T

# Determine most variably expressed genes and subset
num_mad_genes = 5000
mad_genes = rnaseq_df.mad(axis=0).sort_values(ascending=False)
top_mad_genes = mad_genes.iloc[0:num_mad_genes, ].index
rnaseq_subset_df = rnaseq_df.loc[:, top_mad_genes]

In [14]:
mad_genes.head(20) # verify 1. these are gene names, 2. relative functional relevance via ToppGene

TCL1A      1.341202
NFKBIA     1.270869
ACTG1      1.260780
JUN        1.259718
CXCR4      1.238830
IGLL5      1.237369
TSC22D3    1.237021
TXNIP      1.229943
GAPDH      1.222030
JUNB       1.213186
TAGLN2     1.211853
CLIC1      1.208219
MT-ATP6    1.205714
PNRC1      1.196760
MYL6       1.196618
DUSP1      1.194378
ARPC2      1.193596
HLA-DMA    1.189252
ERP29      1.188631
MT-ND5     1.187331
dtype: float64

In [15]:
# Scale RNAseq data using z-scores
rnaseq_scaled_df = preprocessing.StandardScaler().fit_transform(rnaseq_subset_df)
rnaseq_scaled_df = pd.DataFrame(rnaseq_scaled_df,
                                columns=rnaseq_subset_df.columns,
                                index=rnaseq_subset_df.index)
rnaseq_scaled_df.to_csv(rna_out_file, sep='\t', compression='gzip')

# Scale RNAseq data using zero-one normalization
rnaseq_scaled_zeroone_df = preprocessing.MinMaxScaler().fit_transform(rnaseq_subset_df)
rnaseq_scaled_zeroone_df = pd.DataFrame(rnaseq_scaled_zeroone_df,
                                        columns=rnaseq_subset_df.columns,
                                        index=rnaseq_subset_df.index)
rnaseq_scaled_zeroone_df.to_csv(rna_out_zeroone_file, sep='\t', compression='gzip')

In [16]:
# check if the flu genes are included in the top mad genes
[i for i in mad_genes.index if 'RSV' in i]

[]